## Tidy Data Exercises

In [2]:
import pandas as pd
import numpy as np

# Attendance Data
- Load the attendance.csv file 
- Calculate an attendnace percentage for each student
  - One half day is worth 50% of a full day, and 10 tardies is equal to one absence.
  - You should end up with something like this:

| name| |
| :-: | :-: |
|Billy | 0.5250 |
Jane | 0.6875 |
John | 0.9125 |
Sally | 0.7625 |

In [114]:
attendance = pd.read_csv('attendance.csv')
attendance.head()

,Unnamed: 0,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [115]:
attendance = attendance.rename(columns={'Unnamed: 0' : 'name'} )

attendance = attendance.apply(lambda col: col.str.replace('P','1').str.replace('T','0.9').str.replace('H','0.5').str.replace('A','0'))

attendance[['2018-01-01','2018-01-02','2018-01-03','2018-01-04',
            '2018-01-05','2018-01-06','2018-01-07','2018-01-08']] = attendance[['2018-01-01','2018-01-02','2018-01-03',
                                                                                '2018-01-04','2018-01-05','2018-01-06',
                                                                                '2018-01-07','2018-01-08']].apply(pd.to_numeric)

attendance['Percentage'] = attendance[['2018-01-01','2018-01-02','2018-01-03','2018-01-04',
                                        '2018-01-05','2018-01-06','2018-01-07','2018-01-08']].apply(np.sum,axis=1).apply(lambda x: x/8)

In [119]:
attendance[['name','Percentage']].sort_values(by='name')

,name,Percentage
2,Billy,0.5250
1,Jane,0.6875
3,John,0.9125
0,Sally,0.7625


In [ ]:
#### Tried another way

In [158]:
attendance = pd.read_csv('attendance.csv')
attendance = attendance.rename(columns={'Unnamed: 0' : 'name'})

In [159]:
attendance = attendance.melt(id_vars=['name'], var_name='dates',value_name='attendance')

attendance['attendance'] = (attendance['attendance'].str.replace('P','1')
                            .str.replace('T','0.9').str.replace('H','0.5')
                            .str.replace('A','0').apply(pd.to_numeric))

attendance.groupby('name').agg('sum').apply(lambda x: x / 8)

,attendance
name,
Billy,0.5250
Jane,0.6875
John,0.9125
Sally,0.7625


# Coffee Levels
- Read the coffee_levels.csv file.
- Transform the data so that each carafe is in it's own column.
- Is this the best shape for the data?

In [121]:
coffee = pd.read_csv('coffee_levels.csv')
coffee.head()

,hour,coffee_carafe,coffee_amount
0,8,x,0.816164
1,9,x,0.451018
2,10,x,0.843279
3,11,x,0.335533
4,12,x,0.898291


In [152]:
coffee = coffee.pivot(index='hour', columns='coffee_carafe')
coffee

coffee_amount                    
coffee_carafe             x         y         z
hour                                           
8                  0.816164  0.189297  0.999264
9                  0.451018  0.521502  0.915990
10                 0.843279  0.023163  0.144928
11                 0.335533  0.235529  0.311495
12                 0.898291  0.017009  0.771947
13                 0.310711  0.997464  0.398520
14                 0.507288  0.058361  0.864464
15                 0.215043  0.144644  0.436364
16                 0.183891  0.544676  0.280621
17                 0.391560  0.594126  0.436677

- This shape does make the data more readable for humans when comparing the coffee amount per hour for each seperate carafe. As far as accessibility with pandas the data would be better off left in it's original form.

# Cake Recipes
- Read the cake_recipes.csv data. 
- This data set contains cake tastiness scores for combinations of different recipes, oven rack positions, and oven temperatures.
- 1) Tidy the data as necessary.
- 2) Which recipe, on average, is the best? (recipe b)
- 3) Which oven temperature, on average, produces the best results? (275)
- 4) Which combination of recipe, rack position, and temperature gives the best result? (recipe b, bottom rack, 300 degrees)



In [197]:
#1
cake = pd.read_csv('cake_recipes.csv')
cake_melt = cake.melt(id_vars=['recipe:position'],var_name='temp',value_name='tastiness_score')
re_po = cake_melt['recipe:position'].str.split(':', expand=True)
re_po.columns = ['recipe','position']
cake_2 = pd.concat([re_po,cake_melt],axis=1).drop(columns='recipe:position')
cake_tidy = cake_2.pivot_table(index=['recipe','position','temp'],values='tastiness_score')
cake_tidy.columns.name = ''
cake_tidy.reset_index(inplace=True)
cake_tidy.head()

,recipe,position,temp,tastiness_score
0,a,bottom,225,61.738655
1,a,bottom,250,53.912627
2,a,bottom,275,74.414730
3,a,bottom,300,98.786784
4,a,top,225,51.709751


In [225]:
#2
cake_tidy.groupby('recipe').agg('mean')
# Recipe B looks to have the best average

,tastiness_score
recipe,
a,63.922201
b,76.736074
c,75.874748
d,62.864844


In [204]:
#3
cake_tidy.groupby('temp').agg('mean')
# Temp of 275 looks to have the best average

,tastiness_score
temp,
225,71.306022
250,66.577437
275,74.886754
300,66.627655


In [217]:
#4
cake_tidy[cake_tidy.tastiness_score == cake_tidy.tastiness_score.agg('max')]
# Recipe B on the bottom rack at 300 degrees looks to have the highest tastiness score.

,recipe,position,temp,tastiness_score
11,b,bottom,300,99.248541
